In [26]:
from ultralytics import YOLO
import numpy as np
import torch
import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from dataset import DetectionDataset
from mean_average_precision import MetricBuilder
import gc
import pandas as pd
import json
import os
import cv2
from PIL import Image
import  matplotlib.pyplot as plt
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import transforms
import seaborn as sns
import albumentations as A
import sys
import tqdm
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset

In [4]:
model = YOLO("yolov8x.pt")

In [16]:
dataset_path = "/seminar_objdet_retina_oi5_ball/data.yaml"

In [27]:
class DetectionDataset(Dataset):
    def __init__(self, data_dict_file, transforms=None, add_path=''):
        with open(data_dict_file, 'r') as f:
            self.data_dict = json.load(f)
        self.transforms = transforms
        self.add_path = add_path
        self.image_files = list(self.data_dict.keys())

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.add_path, self.image_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        bboxes = self.data_dict[self.image_files[idx]]
        labels = [0] * len(bboxes)  # Предполагаем, что все bounding boxes относятся к одному классу (например, мячу)

        sample = {'image': image, 'bboxes': bboxes, 'labels': labels}

        if self.transforms:
            sample = self.transforms(**sample)

        return sample

In [28]:
# Define transformations
transform_train = A.Compose([
    A.LongestMaxSize(620),
    A.SmallestMaxSize(520),
    A.RandomCrop(height=400, width=400),
    A.HorizontalFlip(p=0.5),
    A.MotionBlur(blur_limit=17, p=0.2),
    A.RandomBrightnessContrast(p=0.2),
    A.ShiftScaleRotate(p=0.2),
    A.RandomBrightnessContrast(p=0.3),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', min_visibility=0.3, label_fields=['labels']))

transform_val = A.Compose([
    A.LongestMaxSize(620),
    A.SmallestMaxSize(520),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))

In [29]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [30]:
train_dataset = DetectionDataset(
    data_dict_file="./seminar_objdet_retina_oi5_ball/oi5_ball_filename_to_bbox_train.json",
    transforms=transform_train,
    add_path='./seminar_objdet_retina_oi5_ball/oi5_ball'
)
train_dataloader = DataLoader(train_dataset, collate_fn=collate_fn, batch_size=4)

val_dataset = DetectionDataset(
    data_dict_file="./seminar_objdet_retina_oi5_ball/oi5_ball_filename_to_bbox_val.json",
    transforms=transform_val,
    add_path='./seminar_objdet_retina_oi5_ball/oi5_ball'
)
val_dataloader = DataLoader(val_dataset, collate_fn=collate_fn, batch_size=4)

In [34]:
def routine(model, train_loader, val_loader, epochs=1):
    for epoch_num in range(epochs):
        # Create a temporary data.yaml file in memory
        data_yaml = f"""
        train: {train_loader.dataset.add_path}
        val: {val_loader.dataset.add_path}

        nc: 1  # number of classes
        names: ['ball']  # list of class names
        """
        with open('/tmp/data.yaml', 'w') as f:
            f.write(data_yaml)

        model.train(data='/tmp/data.yaml', epochs=1, batch=16)
        val_step(model, val_loader)

In [32]:
def val_step(model, dataloader):
    model.eval()
    metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=False, num_classes=1)
    for iter_num, data in tqdm.tqdm(enumerate(dataloader), total=len(dataloader)):
        for img, gt in zip(data['image'], data['bboxes']):
            with torch.no_grad():
                results = model(img.unsqueeze(0))
                preds = results.pred[0].cpu().detach().numpy()
            zeros = torch.zeros((gt.shape[0], 1))
            gt = torch.cat([gt, zeros, zeros], 1).cpu().detach().numpy()
            metric_fn.add(preds, gt)

            del img, gt, preds, zeros, results
        gc.collect()
        torch.cuda.empty_cache()
    print(f"validation mAP in all points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")


In [35]:
routine(model, train_dataloader, val_dataloader, epochs=1)

New https://pypi.org/project/ultralytics/8.3.31 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145 🚀 Python-3.7.3 torch-1.13.1+cu117 CUDA:0 (Tesla V100-SXM3-32GB, 32510MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/tmp/data.yaml, epochs=1, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualiz

RuntimeError: Dataset '/tmp/data.yaml' error ❌ 
Dataset '/tmp/data.yaml' images not found ⚠️, missing path '/tmp/seminar_objdet_retina_oi5_ball/oi5_ball'
Note dataset download directory is '/raid/alebedev/Technopark/ANN/dl-course/lectures/lecture6/detection/datasets'. You can update this in '/raid/alebedev/.config/Ultralytics/settings.yaml'